In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import torch 
from spanemo.model import SpanEmo
from spanemo.data_loader import DataClass, twitter_preprocessor, alternate_preprocessor
from fastprogress.fastprogress import format_time, master_bar, progress_bar
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import datetime
import numpy as np
import json
import math
import random

In [2]:
seed = 12345678
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

if str(device) == 'cuda:0':
    print("Currently using GPU: {}".format(device))
else:
    print("WARNING: USING CPU")

Currently using GPU: cuda:0


In [3]:
# pred_df = pd.read_csv("data.csv")
# pred_df = pred_df.loc[:, ["text", "conversation", "source"]]
# pred_df = pred_df.reindex(columns=pred_df.columns.tolist() + [str(x) for x in range(11)])
# pred_df.to_csv("predictions.csv", index=False)

In [4]:
pred_df = pd.read_csv("predictions.csv")
pred_idx = pred_df.groupby("conversation").filter(lambda x: x["text"].count() >= 5)
pred_idx = pred_idx.groupby("conversation").tail(-4).index.to_series()
pred_text = pred_idx.apply(lambda x: pred_df.loc[x-4:x+1, "text"].values).values

/tmp/ipykernel_325768/2574911869.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  pred_df = pd.read_csv("predictions.csv")


In [5]:
hyperparams = {
    'backbone':'bert-base-uncased',
    'max_length': 512, #maximum context length for the model
    'batch_size': 32,
}

In [6]:
def loop(start=0, x=32768):
    model_path = "models/final_model.pt"
    model = SpanEmo(backbone=hyperparams["backbone"])
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()

    preprocessor = alternate_preprocessor()
    
    mb = master_bar(range(start, math.ceil(len(pred_idx)/x)))
    for i in mb:
        dataset = DataClass(hyperparams, 
                            pred_text[i*x : min(i*x+x, len(pred_idx))], 
                            pred_mode=True, 
                            preprocessor=preprocessor,
                            pbar=lambda x, **kw: progress_bar(x, parent=mb, leave=False))
        data_loader = DataLoader(dataset,
                                batch_size=hyperparams['batch_size'],
                                shuffle=False
                                )
        
        with torch.no_grad():
            current_index = 0
            for step, batch in enumerate(progress_bar(data_loader, parent=mb, leave=False)):
                num_rows, y_pred, logits = model.predict(batch, device)

                pred_df.loc[pred_idx[i*x + current_index: i*x + current_index + num_rows], "0":"10"] = logits.cpu().numpy()
                if step % 16 == 0 and step > 0: pred_df.to_csv("predictions.csv", index=False)
                
                current_index += num_rows

        pred_df.to_csv("predictions.csv", index=False)

In [7]:
loop(start=7)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("

Reading english - 1grams ...
Reading english - 2grams ...
Reading english - 1grams ...


/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in
